# Hatespeech research
This script is made to clean and aggregate the data automatically. 
Script environment is python with R extension.
You need to upload the data to your Google drive and connect.

Firstly I'll load the R environment from Python runtime. Mount Google drive and install the necessary packages and load them.

In [0]:
%load_ext rpy2.ipython
#Load R extension

In [0]:
from google.colab import drive #Mount G-drive
drive.mount('/content/drive')

In [0]:
%%R
library (tidyverse)
install.packages(c("openxlsx","data.table"))
library (openxlsx)
library (data.table)

* Filter and sample the data from the CrowdTangle. 
* CTHP data preparation

In [0]:
%%R
master_dir <- "/content/drive/'My Drive'/hatespeech/CTHP/"
Today <- format(Sys.time(), "D%dM%m")
system("mkdir /content/hs")
cdir <- "/content/hs/"
system(paste0("cp ",master_dir,Today,".csv ",cdir))
CTHP <- read.csv (paste0(cdir,Today,".csv"), stringsAsFactors = F)
CTHP <- CTHP %>% filter (Comments >= 100) %>% mutate (prob = Comments/sum(Comments))
if (nrow(CTHP) > 100) {
    lexicon <- read.csv ("https://docs.google.com/spreadsheets/d/e/2PACX-1vTyhO1LfxmVwkA_6QJnGRlV0Y0269WMcOnJymkA6ropJZH5i29Ob56rNuAWYwv4aAakXjsiqOBET2Gn/pub?gid=2071343638&single=true&output=csv", stringsAsFactors = F)
    lexicon <- unlist(lexicon)
    hs_row <- NULL
    for (i in 1:nrow(CTHP)) {
        hs_row[i] <- any(str_detect(CTHP[i,"Description"],lexicon))
    }
    hs_row[is.na(hs_row)] <- F
    if (sum(hs_row) > 100) {
        CTHP <- CTHP[hs_row,]
        CTHP <- CTHP[sample(1:nrow(CTHP),100, prob = CTHP$prob),]
    } else {
        CTU <- CTHP[!hs_row,]
        CTHP <- rbind.data.frame(CTU[sample(1:nrow(CTU),(100-sum(hs_row)),prob = CTU$prob),],CTHP[hs_row,])
    }
}
system("mkdir sample")
write.csv(CTHP, paste0("./sample/",Today,".csv"), row.names = F)
system(paste0("mv ./sample/",Today,".csv /content/drive/'My Drive'/hatespeech/CTHP/sample/"))
system(paste0("rm ",cdir,Today,".csv"))

This section copy the data from the Google drive to the local runtime. (This step is crucial because we cannot load the data directly from the Google drive).

In [0]:
%%R
wdir <- paste0("./drive/'My Drive'/hatespeech/Post/", Today)
system(paste0("cp ", wdir, "/* ", cdir))
item_list <- system(paste("ls",cdir), intern = T)
TD_data <- data.table()
smp_func <- function (s = 5) {
  for (i in 1:length(item_list)) {
    assign(paste0("docs",i), read.xlsx(paste0(cdir,item_list[i]), sheet = 1, startRow = 6, colNames = T, detectDates = T))
    unirl <- read.xlsx(paste0(cdir,item_list[i]), sheet = 1, rows = 2, cols = 2, colNames = F)
    unirl <- unlist(unirl)
    temp_df <- eval(as.name(paste0("docs",i)))
    temp_df <- temp_df[sample(nrow(temp_df),s, prob = temp_df$prob),]
    temp_df$url <- unirl
    TD_data <- rbind.data.frame(TD_data,temp_df)
    rm(temp_df)
  }
  return(TD_data)
}
if (length(item_list)< 100) {
    s = ceiling(500/length(item_list))
    TD_data <- smp_func(s = s)
    TD_data <- TD_data [sample(nrow(TD_data), 500),]
} else {
  TD_data <- smp_func()
}

write.csv(TD_data, paste0(cdir,"C",Today,".csv"), row.names = F)
system(paste0("mv ",cdir,"C",Today,".csv /content/drive/'My Drive'/hatespeech/Post/"))